In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
from scipy import signal
import tensorflow as tf
import matplotlib.pyplot as plt
from utilities import *
from features import *

In [ ]:
#cross validation method
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from itertools import cycle
from scipy import interp
from itertools import chain

#model
from sklearn import svm
from sklearn import neighbors
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.preprocessing import label_binarize
from tensorflow.python.ops import rnn, rnn_cell

Readme:<br/>

The first two folds, 'Read whole data...' and 'Feature extraction' can be skipped if you do not want to change the window size, filter frequency and features. The default window size is setting as 10, emg filter with 20 Hz cutoff frequency, and 12 type of features.<br/>

However, if you want to change any of these parameters, you should open first two folds and check the utilities.py or features.py to modify them. <br/>

Therefore, you can go straight to the third fold, 'Data Spliting'. Run the first cell inside this fold to read the features you want to test.

### Read whole data and store them in seperate files

#### This cell code is store the dataset after filter and fft process, and they are already exsit in the data files.

   So it is not necessary to run this cell unless the datasets have been deleted or the filter frequency want to be changed.

In [ ]:
for j in range (1,33):
    file_prefix = './data/Trial'
    acc_df = pd.read_csv(file_prefix + str(j) + '/Myo_accelerometer.csv',index_col=None, header=None)

    acc_temp = acc_df.values
    acc_dataOut = np.zeros((len(acc_temp),3))
    acc_dataOut_fft = np.zeros((len(acc_temp),3))
    for p in range(3):
        acc_dataOut[:,p] = butter_lowpass_filter(acc_df.iloc[:,p+1], cutoff, fss, order) #cutoff
        acc_dataOut[:,p] = acc_dataOut[:,p]/max(abs(acc_dataOut[:,p]))
        acc_dataOut_fft[:,p] = np.fft.fft(acc_dataOut[:,p]).real

    acc_data_filter = np.concatenate([acc_temp[:,0:1], acc_dataOut], axis=1)
    acc_data_filter_fft = np.concatenate([acc_temp[:,0:1], acc_dataOut_fft], axis=1)

    acc_df = pd.DataFrame.from_records(acc_data_filter)
    acc_df_fft = pd.DataFrame.from_records(acc_data_filter_fft)



    gyro_df = pd.read_csv(file_prefix + str(j) + '/Myo_gyro.csv',index_col=None, header=None)

    gyro_temp = gyro_df.values
    gyro_dataOut = np.zeros((len(gyro_temp),3))
    gyro_dataOut_fft = np.zeros((len(gyro_temp),3))
    for p in range(3):
        gyro_dataOut[:,p] = butter_lowpass_filter(gyro_df.iloc[:,p+1], cutoff, fss, order)
        gyro_dataOut[:,p] = gyro_dataOut[:,p]/max(abs(gyro_dataOut[:,p]))
        gyro_dataOut_fft[:,p] = np.fft.fft(gyro_dataOut[:,p]).real

    gyro_data_filter = np.concatenate([gyro_temp[:,0:1], gyro_dataOut], axis=1)
    gyro_data_filter_fft = np.concatenate([gyro_temp[:,0:1], gyro_dataOut_fft], axis=1)

    gyro_df = pd.DataFrame.from_records(gyro_data_filter)
    gyro_df_fft = pd.DataFrame.from_records(gyro_data_filter_fft)


    euler_df = pd.read_csv(file_prefix + str(j) + '/Myo_orientationEuler.csv',index_col=None, header=None)

    euler_temp = euler_df.values
    euler_dataOut = np.zeros((len(euler_temp),3))
    euler_dataOut_fft = np.zeros((len(euler_temp),3))
    for p in range(3):
        euler_dataOut[:,p] = butter_lowpass_filter(euler_df.iloc[:,p+1], cutoff, fss, order)
        euler_dataOut[:,p] = euler_dataOut[:,p]/max(abs(euler_dataOut[:,p]))
        euler_dataOut_fft[:,p] = np.fft.fft(euler_dataOut[:,p]).real

    euler_data_filter = np.concatenate([euler_temp[:,0:1], euler_dataOut], axis=1)
    euler_data_filter_fft = np.concatenate([euler_temp[:,0:1], euler_dataOut_fft], axis=1)


    euler_df = pd.DataFrame.from_records(euler_data_filter)
    euler_df_fft = pd.DataFrame.from_records(euler_data_filter_fft)


    emg_df = pd.read_csv(file_prefix + str(j) + '/Myo_emg.csv',index_col=None, header=None)

    temp = emg_df.values
    dataOut = np.zeros((len(temp),8))
    dataOut_fft = np.zeros((len(temp),8))
    for p in range(8):
        dataOut[:,p] = emg_butter_lowpass_filter(emg_df.iloc[:,p+1], emg_cutoff, fs, order) # ,emg_cutoff
        dataOut[:,p] = dataOut[:,p]/max(abs(dataOut[:,p]))
        dataOut_fft[:,p] = np.fft.fft(dataOut[:,p]).real

    data_filter = np.concatenate([temp[:,0:1], dataOut], axis=1)
    data_filter_fft = np.concatenate([temp[:,0:1], dataOut_fft], axis=1)

    emg_df = pd.DataFrame.from_records(data_filter)
    emg_df_fft = pd.DataFrame.from_records(data_filter_fft)

    emg_df.to_csv(file_prefix + str(j) + '/emg_filter' + '.csv', index=False)
    emg_df_fft.to_csv(file_prefix + str(j) + '/emg_fft' + '.csv', index=False)

    gyro_df.to_csv(file_prefix + str(j) + '/gyro_filter' + '.csv', index=False)
    gyro_df_fft.to_csv(file_prefix + str(j) + '/gyro_fft' + '.csv', index=False)

    euler_df.to_csv(file_prefix + str(j) + '/orientationEuler_filter' + '.csv', index=False)
    euler_df_fft.to_csv(file_prefix + str(j) + '/orientationEuler_fft' + '.csv', index=False)

    acc_df.to_csv(file_prefix + str(j) + '/accelerometer_filter' + '.csv', index=False)
    acc_df_fft.to_csv(file_prefix + str(j) + '/accelerometer_fft' + '.csv', index=False)

#### Only Filtered Dataset (This cell should be re-run every time when the labels are changed)

resample and segement window

In [ ]:
"""" read and label the datasets of only Filter data """

name = ['gyro_filter','orientationEuler_filter','accelerometer_filter']
gyro = store_label_data(name[0])
Myo_orientationEuler = store_label_data(name[1])
Myo_accelerometer = store_label_data(name[2])
imu_data = pd.concat([gyro.iloc[:,0:4],Myo_orientationEuler.iloc[:,1:4],Myo_accelerometer.iloc[:,1:5]],axis=1)
imu_data = imu_data.iloc[:,1:]
name = ['emg_filter']
emg = store_label_data(name[0])
emg_data = emg.iloc[:,1:]

"""" resample the EMG data into the same length of IMU data saved as resample_data """
f0 = signal.resample(emg_data.iloc[:,0], len(imu_data)) # resample data to imu numbers
f1 = signal.resample(emg_data.iloc[:,1], len(imu_data))
f2 = signal.resample(emg_data.iloc[:,2], len(imu_data))
f3 = signal.resample(emg_data.iloc[:,3], len(imu_data))
f4 = signal.resample(emg_data.iloc[:,4], len(imu_data))
f5 = signal.resample(emg_data.iloc[:,5], len(imu_data))
f6 = signal.resample(emg_data.iloc[:,6], len(imu_data))
f7 = signal.resample(emg_data.iloc[:,7], len(imu_data))
resample_data = np.concatenate([f0[:,np.newaxis], f1[:,np.newaxis], f2[:,np.newaxis],f3[:,np.newaxis],
                                f4[:,np.newaxis], f5[:,np.newaxis], f6[:,np.newaxis], f7[:,np.newaxis]],axis=1)
resample_data = pd.DataFrame(resample_data)

In [ ]:
"""" set the window size and segement IMU and EMG respectively  """
window_size = 10

# get the label and the label after encoding in one hot 
labels = extract_labels(imu_data,window_size)
onehot_labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

##############      IMU         #################
data_len = imu_data.iloc[:,0].shape[0]
reshaped_x1 = imu_data.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = imu_data.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = imu_data.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = imu_data.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = imu_data.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = imu_data.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = imu_data.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = imu_data.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_x9 = imu_data.iloc[:(data_len//window_size)*window_size,8].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_imu = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8, reshaped_x9],axis=2)

##############      EMG        #################
data_len = resample_data.iloc[:,0].shape[0]
reshaped_x1 = resample_data.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = resample_data.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = resample_data.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = resample_data.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = resample_data.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = resample_data.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = resample_data.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = resample_data.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_emg = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8],axis=2)

In [ ]:
# check the shape of each data and labels
reshaped_segments_imu.shape,reshaped_segments_emg.shape,labels.shape,onehot_labels.shape

#### Filtered and FFT Dataset (This cell should be re-run every time when the labels are changed)

In [ ]:
"""" read and label the datasets of Filtered and FFT data """

name = ['gyro_fft','orientationEuler_fft','accelerometer_fft']
gyro_fft = store_label_data(name[0])
Myo_orientationEuler_fft = store_label_data(name[1])
Myo_accelerometer_fft = store_label_data(name[2])
imu_data_fft = pd.concat([gyro_fft.iloc[:,0:4],Myo_orientationEuler_fft.iloc[:,1:4],Myo_accelerometer_fft.iloc[:,1:5]],axis=1)
imu_data_fft = imu_data_fft.iloc[:,1:]
name = ['emg_fft']
emg_fft = store_label_data(name[0])
emg_data_fft = emg_fft.iloc[:,1:]

"""" resample the EMG data into the same length of IMU data saved as resample_data """
f0 = signal.resample(emg_data_fft.iloc[:,0], len(imu_data_fft)) # resample data to imu numbers
f1 = signal.resample(emg_data_fft.iloc[:,1], len(imu_data_fft))
f2 = signal.resample(emg_data_fft.iloc[:,2], len(imu_data_fft))
f3 = signal.resample(emg_data_fft.iloc[:,3], len(imu_data_fft))
f4 = signal.resample(emg_data_fft.iloc[:,4], len(imu_data_fft))
f5 = signal.resample(emg_data_fft.iloc[:,5], len(imu_data_fft))
f6 = signal.resample(emg_data_fft.iloc[:,6], len(imu_data_fft))
f7 = signal.resample(emg_data_fft.iloc[:,7], len(imu_data_fft))
resample_data_fft = np.concatenate([f0[:,np.newaxis], f1[:,np.newaxis], f2[:,np.newaxis],f3[:,np.newaxis],
                                f4[:,np.newaxis], f5[:,np.newaxis], f6[:,np.newaxis], f7[:,np.newaxis]],axis=1)
resample_data_fft = pd.DataFrame(resample_data_fft)

In [ ]:
window_size = 10

##############      IMU         #################
data_len = imu_data_fft.iloc[:,0].shape[0]
reshaped_x1 = imu_data_fft.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = imu_data_fft.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = imu_data_fft.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = imu_data_fft.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = imu_data_fft.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = imu_data_fft.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = imu_data_fft.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = imu_data_fft.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_x9 = imu_data_fft.iloc[:(data_len//window_size)*window_size,8].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_imu_fft = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8, reshaped_x9],axis=2)

##############      EMG        #################
data_len = resample_data_fft.iloc[:,0].shape[0]
reshaped_x1 = resample_data_fft.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = resample_data_fft.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = resample_data_fft.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = resample_data_fft.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = resample_data_fft.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = resample_data_fft.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = resample_data_fft.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = resample_data_fft.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_emg_fft = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8],axis=2)

In [ ]:
reshaped_segments_imu_fft.shape,reshaped_segments_emg_fft.shape

### Feature extraction
sma, mean, std, entropy, rms, zc, skew, kurt, pfd, meanfft, binpower

In [ ]:
itr = reshaped_segments_imu.shape[0]
emg_sma = np.zeros((itr,1))

emg_mean = np.zeros((itr,8))
emg_zc = np.zeros((itr,8))
emg_std = np.zeros((itr,8))
emg_entropy = np.zeros((itr,8))
emg_pfd = np.zeros((itr,8))
emg_rms = np.zeros((itr,8))
emg_kurt = np.zeros((itr,8))
# emg_energy = np.zeros((itr,8))
emg_skew = np.zeros((itr,8))

a_emg_get_bin_power = np.zeros((itr,8))
b_emg_get_bin_power = np.zeros((itr,8))
emg_mean_fft = np.zeros((itr,8))
emg_energy_fft = np.zeros((itr,8))

ea_band = [1,8]
eb_band = [9,16]

label = np.zeros((itr,1))

for i in range(itr):
    emg_sma[i,:] = get_sma_numpy(reshaped_segments_emg[i,:,:])
    for k in range(8):
        emg_mean[i,k] = get_mean(reshaped_segments_emg[i,:,k])
        emg_std[i,k] = get_std(reshaped_segments_emg[i,:,k]) # std 第二个 9-16 和 第四个 rms 25-32 和 第6个 41-48类似
        emg_entropy[i,k] = get_entropy(reshaped_segments_emg[i,:,k]) # 
        emg_rms[i,k] = get_RMS(reshaped_segments_emg[i,:,k]) # std, rms 非常相似， 和skew有一部分相似
        emg_zc[i,k] = get_ZC(reshaped_segments_emg[i,:,k]) # 17-24 and 33-40 nan
        emg_skew[i,k] = get_skew(reshaped_segments_emg[i,:,k])
        emg_kurt[i,k] = get_kurt(reshaped_segments_emg[i,:,k])
        emg_pfd[i,k] = get_pfd(reshaped_segments_emg[i,:,k])
           
        emg_mean_fft[i,k] = get_mean(reshaped_segments_emg_fft[i,:,k])
#         emg_energy_fft[i,k] = get_energy(reshaped_segments_emg_fft[i,:,k])
        a_emg_get_bin_power[i,k] = get_bin_power(reshaped_segments_emg_fft[i,:,k],ea_band)
        b_emg_get_bin_power[i,k] = get_bin_power(reshaped_segments_emg_fft[i,:,k],eb_band)  
        
emg_features = np.concatenate([emg_sma,emg_mean,emg_std,emg_entropy,emg_rms,emg_zc,emg_skew,emg_kurt,
                               emg_pfd,emg_mean_fft,a_emg_get_bin_power,b_emg_get_bin_power], axis=1)

In [ ]:
itr = reshaped_segments_imu.shape[0]
gyro_sma = np.zeros((itr,1))
orien_sma = np.zeros((itr,1))
acc_sma = np.zeros((itr,1))
imu_mean = np.zeros((itr,9))
imu_std = np.zeros((itr,9))
imu_pfd = np.zeros((itr,9))
imu_rms = np.zeros((itr,9))
imu_entropy = np.zeros((itr,9))
imu_kurt = np.zeros((itr,9))
imu_skew = np.zeros((itr,9))
# imu_energy = np.zeros((itr,9))
imu_zc = np.zeros((itr,9))

a_imu_get_bin_power = np.zeros((itr,9))
b_imu_get_bin_power = np.zeros((itr,9))
imu_energy_fft = np.zeros((itr,9))
imu_mean_fft = np.zeros((itr,9))
label = np.zeros((itr,1))
a_band = [1,8]
b_band = [9,16]

for i in range(itr):
    gyro_sma[i,:] = get_sma_numpy(reshaped_segments_imu[i,:,:3])
    orien_sma[i,:] = get_sma_numpy(reshaped_segments_imu[i,:,3:6])
    acc_sma[i,:] = get_sma_numpy(reshaped_segments_imu[i,:,6:9]) # 0-2
    for k in range(9):
        imu_mean[i,k] = get_mean(reshaped_segments_imu[i,:,k]) # 3-12
        imu_std[i,k] = get_std(reshaped_segments_imu[i,:,k]) # 
        imu_entropy[i,k] = get_entropy(reshaped_segments_imu[i,:,k]) # (21-29)and (39-47) 
        imu_rms[i,k] = get_RMS(reshaped_segments_imu[i,:,k]) # 这个feature 和imu_energy相似 (抛弃30-38)
        imu_zc[i,k] = get_ZC(reshaped_segments_imu[i,:,k]) # zero和entropy存在nan的correlation关系 不知道为什么
        imu_skew[i,k] = get_skew(reshaped_segments_imu[i,:,k]) 
        imu_kurt[i,k] = get_kurt(reshaped_segments_imu[i,:,k])
        imu_pfd[i,k] = get_pfd(reshaped_segments_imu[i,:,k])
        
        imu_mean_fft[i,k] = get_mean(reshaped_segments_imu_fft[i,:,k]) # 12-21
#         imu_energy_fft[i,k] = get_energy(reshaped_segments_imu_fft[i,:,k]) # 这个在SFS中被消除了，不需要这个feature.
        a_imu_get_bin_power[i,k] = get_bin_power(reshaped_segments_imu_fft[i,:,k],a_band)
        b_imu_get_bin_power[i,k] = get_bin_power(reshaped_segments_imu_fft[i,:,k],b_band)
        
        
imu_features = np.concatenate([gyro_sma,orien_sma,acc_sma,imu_mean,imu_std,imu_entropy,imu_rms,imu_zc,
                               imu_skew,imu_kurt,imu_pfd,imu_mean_fft,a_imu_get_bin_power,
                               b_imu_get_bin_power], axis=1)

In [ ]:
# concatenate emg and imu features
all_features = np.concatenate([emg_features,imu_features],axis=1)
df_emg_features = pd.DataFrame(emg_features)
df_imu_features = pd.DataFrame(imu_features)
df_all_features = pd.DataFrame(all_features)
df_onehot_labels = pd.DataFrame(onehot_labels)
df_labels = pd.DataFrame(labels)


"""" If you want to store the features for convenience, then uncomment the three lines code below"""
df_emg_features.to_csv('data/emg_features' + '.csv', index=False, header=None)
df_imu_features.to_csv('data/imu_features' + '.csv', index=False, header=None)
df_all_features.to_csv('data/all_features' + '.csv', index=False, header=None)
df_onehot_labels.to_csv('data/onehotlabels' + '.csv', index=False, header=None)
df_labels.to_csv('data/lables' + '.csv', index=False, header=None)

### Data Spliting (IMU, EMG, Fusion)

Here, it should take care of the labels format (one hot or decimal). <br/>

one hot for LSTM model<br/>
decimal for Classical models

In [ ]:
emg_features = pd.read_csv('data/emg_features.csv',index_col=None, header=None).values
imu_features = pd.read_csv('data/imu_features.csv',index_col=None, header=None).values
all_features = pd.read_csv('data/all_features.csv',index_col=None, header=None).values
onehot_labels = pd.read_csv('data/onehotlabels.csv',index_col=None, header=None).values
labels = pd.read_csv('data/lables.csv',index_col=None, header=None).values

#### IMU features

In [ ]:
## splite the data into train and  test sets
train_test_split_value = np.random.rand(len(imu_features)) < 0.8
train_x = imu_features[train_test_split_value]
test_x = imu_features[~train_test_split_value]

"""" one hot labels is used for the LSTM model """
train_y = onehot_labels[train_test_split_value]
test_y = onehot_labels[~train_test_split_value]

"""" original labels is used for the other classical models """
class_train_y = labels[train_test_split_value]
class_test_y = labels[~train_test_split_value]

## splite the train set into train and  validation sets
X_train, X_vld, y_train, y_vld = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_x.shape, X_vld.shape, test_x.shape, onehot_labels.shape, labels.shape

#### EMG features 

In [ ]:
train_test_split_value = np.random.rand(len(emg_features)) < 0.8
train_x = emg_features[train_test_split_value]
test_x = emg_features[~train_test_split_value]

"""" one hot labels is used for the LSTM model """
train_y = onehot_labels[train_test_split_value]
test_y = onehot_labels[~train_test_split_value]

"""" original labels is used for the other classical models """
class_train_y = labels[train_test_split_value]
class_test_y = labels[~train_test_split_value]

X_train, X_vld, y_train, y_vld = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_x.shape, X_vld.shape, test_x.shape, onehot_labels.shape, labels.shape

#### Fusion features

In [ ]:
train_test_split_value = np.random.rand(len(all_features)) < 0.8
train_x = all_features[train_test_split_value]
test_x = all_features[~train_test_split_value]

"""" one hot labels is used for the LSTM model """
train_y = onehot_labels[train_test_split_value]
test_y = onehot_labels[~train_test_split_value]

"""" original labels is used for the other classical models """
class_train_y = labels[train_test_split_value]
class_test_y = labels[~train_test_split_value]

X_train, X_vld, y_train, y_vld = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_x.shape, X_vld.shape, test_x.shape, onehot_labels.shape, labels.shape

## Models

### LSTM

In [ ]:
seq_len = 1
r_len = X_train.shape[0]
c_len = X_train.shape[1]
X_train = X_train[:(r_len//seq_len)*seq_len,:].reshape([r_len//seq_len,seq_len,c_len])

In [ ]:
r_len = X_vld.shape[0]
c_len = X_vld.shape[1]
X_vld = X_vld[:(r_len//seq_len)*seq_len,:].reshape([r_len//seq_len,seq_len,c_len])

In [ ]:
r_len = test_x.shape[0]
c_len = test_x.shape[1]
test_x = test_x[:(r_len//seq_len)*seq_len,:].reshape([r_len//seq_len,seq_len,c_len])
X_train.shape, X_vld.shape, test_x.shape

When build the LSTM graph, it needs check the features 'n_channels' and 'n_classes'<br/>

The default 'n_channels' is 191 features dimensions, and 'n_classes' is 4 for four classes classifiers, which corresponding to the 'Fusion features' cell from the above 'Data Spliting' fold.<br/>

Then when you choose 'EMG features' from the above, you should also the the grapg setting to 'n_channels = 89' and 'lstm_size = 89*3', same for 'IMU'.

In [ ]:
lstm_size = 191*3         # 3 times the amount of channels
lstm_layers = 3        # Number of layers
batch_size = 900      # Batch size
seq_len = 1 # 60         window size  # Number of steps / how many input samples  
learning_rate = 0.0001  # Learning rate (default is 0.001)
epochs = 100

# Fixed
n_classes = 4        # number of classifiers
n_channels = 191         # dimensions of input


graph = tf.Graph()
# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

    # Construct the LSTM inputs and LSTM cells
    lstm_in = tf.transpose(inputs_, [1,0,2]) # reshape into (seq_len, N, channels)
    lstm_in = tf.reshape(lstm_in, [-1, n_channels]) # Now (seq_len*N, n_channels)
    
    # To cells
    lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None) # or tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh?
    
    # Open up the tensor into a list of seq_len pieces
    lstm_in = tf.split(lstm_in, seq_len, 0)
    
    # Add LSTM layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    
    outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32,
                                                     initial_state = initial_state)
    
    # We only need the last output tensor to pass into a classifier
    logits = tf.layers.dense(outputs[-1], n_classes, name='logits')
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    #optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost) # No grad clipping
    
    # Grad clipping
    train_op = tf.train.AdamOptimizer(learning_rate_)

    gradients = train_op.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    optimizer = train_op.apply_gradients(capped_gradients)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    y_pred = tf.argmax(logits, 1)
    saver = tf.train.Saver()

In [ ]:
if (os.path.exists('checkpoints') == False):
    !mkdir checkpoints

In [ ]:
validation_acc = []
validation_loss = []
validation_f1score = []
train_f1score = []

train_acc = []
train_loss = []
train_pred = []
train_true = []
tra_true = []


with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    count = 0
    save_count = 0
    
    for e in range(epochs):
        # Initialize 
        state = sess.run(initial_state)
        # Loop over batches
        for x, y in get_batches(X_train, y_train, batch_size):
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, 
                    initial_state : state, learning_rate_ : learning_rate}
            
            loss, _ , state, acc, yt_pred = sess.run([cost, optimizer, final_state, accuracy, y_pred], 
                                             feed_dict = feed)
            
            train_true = np.argmax(y, axis=1)
            tr_f1score = f1_score(train_true, yt_pred, average='micro')
            train_f1score.append(tr_f1score)
            tra_true.append(y)
            train_pred.append(yt_pred)
            train_acc.append(acc)
            train_loss.append(loss)
#             training_f1score.append(np.mean(train_f1score))
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
#             Compute validation loss at every 25 iterations
            if (iteration%25 == 0):
                
                # Initiate for validation set
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                val_f1score = []
                val_acc_ = []
                val_loss_ = []
                val_pred = []
                val_true = []
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0, initial_state : val_state}
                    
                    # Loss
                    loss_v, state_v, acc_v,yv_pred= sess.run([cost, final_state, accuracy, y_pred], feed_dict = feed)
                    
#                     if (acc_v > val_acc_[-1]) & count > 65:
# #                         saver.save(sess,"checkpoints/har-lstm.ckpt")
#                         save_count += 1
                    
                    valid_true = np.argmax(y_v, axis=1)
                    f1score = f1_score(valid_true, yv_pred, average='micro')
                    val_f1score.append(f1score)
                    val_true.append(y_v)
                    val_pred.append(yv_pred)
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                    
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
                validation_f1score.append(np.mean(val_f1score))
            # Iterate 
            iteration += 1
#     saver.save(sess,"checkpoints/har-lstm.ckpt") # 10 imu cutoff,20 emg cutoff, 30 window size, 5 classes
    saver.save(sess,"checkpoints/har-lstm.ckpt")

#### LSTM analysis

plot the confusion matrix,f1-score and auc

In [ ]:
train_pred =list(chain.from_iterable(train_pred))
tra_true =list(chain.from_iterable(tra_true[-2800:]))
tra_true = np.argmax(tra_true, axis=1)
confusion_m = confusion_matrix(tra_true[-2800:], train_pred[-2800:]) 
confusion_m

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 25 == 0], np.array(validation_loss), 'b-')
plt.xlabel("iteration",fontsize=13)
plt.ylabel("Loss",fontsize=13)
plt.title('Plot training and test loss',fontsize=18)
plt.legend(['train', 'validation'], loc='upper right',fontsize=13)
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

# plt.plot(t[0:30], np.array(train_acc)[0:30], 'r-')
plt.plot(t, np.array(train_f1score), 'r-', t[t % 25 == 0], validation_f1score, 'b-')
plt.xlabel("iteration",fontsize=13)
plt.ylabel("F1-score",fontsize=13)
plt.title('Plot F1-score',fontsize=18)
plt.legend(['train', 'validation'], loc='lower right',fontsize=13)
plt.show()

In [ ]:
test_acc = []
test_pred = []
test_f1score = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    
    for x_t, y_t in get_batches(test_x, test_y, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1,
                initial_state: test_state}
        
        batch_acc, test_state, t_pred = sess.run([accuracy, final_state, y_pred], feed_dict=feed)
        
        test_true = np.argmax(y_t, axis=1)
        t_f1score = f1_score(test_true, t_pred, average='micro')
        test_f1score.append(t_f1score)
        test_pred.append(t_pred)
        test_acc.append(batch_acc)
    print("Test F1-score: {:.6f}".format(np.mean(test_f1score)))

In [ ]:
test_true = label_binarize(test_true, classes=[0,1,2,3])
t_pred = label_binarize(t_pred, classes=[0,1,2,3])
n_classes = 4
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_true[:, i], t_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(test_true.ravel(), t_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Uncomment below codes for plot all ROC curves
plt.figure()
# plt.plot(fpr["micro"], tpr["micro"],
#          label='micro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["micro"]),
#          color='deeppink', linestyle=':', linewidth=4)

# plt.plot(fpr["macro"], tpr["macro"],
#          label='macro-average ROC curve (area = {0:0.2f})'
#                ''.format(roc_auc["macro"]),
#          color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, linewidth=4,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
test_len = (test_x.shape[0]//batch_size)*batch_size
test_pred =list(chain.from_iterable(test_pred))
test_true = np.argmax(test_y, axis=1)
confusion_m = confusion_matrix(test_true[:test_len], test_pred)
confusion_m

In [ ]:
plt.matshow(confusion_m, cmap=plt.cm.Blues)

for x in range(len(confusion_m)): #数据标签
    for y in range(len(confusion_m)):
        plt.annotate(confusion_m[x,y], xy=(x, y), horizontalalignment='center', verticalalignment='center')
plt.colorbar() #颜色标签
plt.ylabel('True label') #坐标轴标签
plt.xlabel('Predicted label') #坐标轴标签
plt.show()

### Classical Models

All the models used 5 cross validation folds to aviod overfitting<br/>

The F1-score is calculated by the 'macro', and display the confusion matrix

In [ ]:
# X = train_x  # X = test_x
# biy = class_train_y  # biy = class_test_y

X = test_x
biy = class_test_y
biy = np.squeeze(biy, axis=(1,))

#### Logistic Regression

In [ ]:
Cs=[0.001,0.01,0.1,1,10,100,1000,10000]
bi_train_f1s=[]
bi_test_f1s=[]

max_c_bi=-1
max_test_f1_bi = 0

for c in Cs:
    clf = LogisticRegression(C=c,solver='liblinear')
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_c_bi=c
x = range(len(Cs))
plt.plot(x, bi_train_f1s, 'ro--',label='train F1')
plt.plot(x, bi_test_f1s, 'go--',label='test F1')
plt.xticks(x, Cs)
plt.title('LogisticRegression with liblinear solver')
plt.xlabel('c')
plt.legend();    
plt.show();    

print('best paras for bi levels:')
print('c:'+str(max_c_bi)+' max_test_f1_bi:'+str(max_test_f1_bi))
print('confusion_matrix:')
print(confusion_m)

#### KNN

In [ ]:
As=['euclidean', 'manhattan','chebyshev','minkowski']
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_k='auto'
max_k_bi='auto'
max_test_f1 = 0
max_test_f1_bi = 0
for k in As:
    clf = neighbors.KNeighborsClassifier(n_neighbors = 10 , metric=k)
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_k_bi=k
        
x = range(len(As))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, As)
plt.xlabel('distance matric')
plt.title('KNN for different distance matric')
plt.legend();    
plt.show();

print('best parameters for bi levels:')
print('metric:'+max_k_bi+', max_test_f1_bi:'+str(max_test_f1_bi))
print('confusion_matrix:')
print(confusion_m)

In [ ]:
# adjust the metric method according the best result get from the above cell
As=[10,50,100,150]
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_k='auto'
max_k_bi='auto'
max_test_f1 = 0
max_test_f1_bi = 0
for k in As:
    clf = neighbors.KNeighborsClassifier(n_neighbors = k , metric='manhattan')
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_k_bi=k
        
x = range(len(As))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, As)
plt.xlabel('distance matric')
plt.title('KNN for different distance matric')
plt.legend();    
plt.show();

print('best parameters for bi levels:')
print('metric:'+str(max_k_bi)+', max_test_f1_bi:'+str(max_test_f1_bi))
print('confusion_matrix:')
print(confusion_m)

#### Random Forest

In [ ]:
max_depths=[1,2,3,4,5,6,7,8,9,10]#,11,12,13,14,15,16,17,18,19,20,21]
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_c=-1
max_c_bi=-1
max_test_f1 = 0
max_test_f1_bi = 0
for max_depth in max_depths:
    clf = RandomForestClassifier(max_depth=max_depth,n_estimators=6, random_state=0)
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_c_bi=max_depth
x = range(len(max_depths))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, max_depths)
plt.title('Random Forest for different max_depths')
plt.legend();    
plt.show();    

print('best para for bi levels:')
print('max_depth:'+str(max_c_bi)+' max_test_f1_bi:'+str(max_test_f1_bi))
print('confusion_matrix:')
print(confusion_m)

In [ ]:
# same as KNN, choose the best result get from the above for the 'max_depth'
ns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_n=-1
max_n_bi=-1
max_test_f1 = 0
max_test_f1_bi = 0
for n in ns:
    clf = RandomForestClassifier(max_depth=7,n_estimators=n,random_state=0)
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_n_bi=n
x = range(len(ns))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, ns)
plt.title('Random Forest for different n_estimators')
plt.legend();    
plt.show(); 

print('best para for bi levels:')
print('n:'+str(max_n_bi)+' max_test_f1_bi:'+str(max_test_f1_bi))
print('confusion_matrix:')
print(confusion_m)

#### LDA

In [ ]:
clf = LinearDiscriminantAnalysis()
[train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
print('confusion_matrix:')
print(confusion_m)
print(test_f1)

#### SVM

In [ ]:
#‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
Cs=[0.001,0.01,0.1,0.2]
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_c=-1
max_c_bi=-1
max_test_f1 = 0
max_test_f1_bi = 0
for c in Cs:
    clf = svm.SVC(C=c, class_weight='balanced', kernel='linear', random_state=0)
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_c_bi=c
x = range(len(Cs))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, Cs)
plt.xlabel('c')
plt.title('SVC with linear kernel')
plt.legend();    
plt.show();    

print('best para for bi levels:')
print('c:'+str(max_c_bi)+' max_test_f1_bi:'+str(max_test_f1_bi))

In [ ]:
# non linear model which takes very long time‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
Cs=[0.001,0.01,0.1,1,10,100]
train_f1s=[]
test_f1s=[]
bi_train_f1s=[]
bi_test_f1s=[]
max_c=-1
max_c_bi=-1
max_test_f1 = 0
max_test_f1_bi = 0
for c in Cs:
    clf = svm.SVC(C=c, class_weight='balanced', kernel='sigmoid', random_state=0)
    [train_acc,test_acc,train_f1,test_f1,confusion_m] = getF1Score(X,biy,clf)
    bi_train_f1s.append(train_f1)
    bi_test_f1s.append(test_f1)
    if (test_f1>max_test_f1_bi):
        max_test_f1_bi = test_f1
        max_c_bi=c
x = range(len(Cs))
plt.plot(x, bi_train_f1s, 'ro--',label='binary train F1')
plt.plot(x, bi_test_f1s, 'go--',label='binary test F1')
plt.xticks(x, Cs)
plt.xlabel('c')
plt.title('SVC with sigmoid kernel')
plt.legend();    
plt.show();

print('best para for bi levels:')
print('c:'+str(max_c_bi)+' max_test_f1_bi:'+str(max_test_f1_bi))

### Sequential Feature Selection

In [ ]:
lr = LogisticRegression(C=10,solver='liblinear')
# lr = svm.SVC(C=c, class_weight='balanced', kernel='linear', random_state=0)
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(lr, 
           k_features= 191, 
           forward=True,
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=1)

sfs1 = sfs1.fit(X, biy)

In [ ]:
df = pd.DataFrame.from_dict(sfs1.get_metric_dict()).T
sor = df.sort_values('cv_scores')
sor

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt


fig = plot_sfs(sfs1.get_metric_dict(), kind='std_dev')
# fig(figsize=[12,12])
plt.ylim([0.5, 1])
plt.xlim([20,40])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()